In [1]:
import time
import threading
from forexconnect import fxcorepy, ForexConnect, ResponseListener, Common

class FC:
    def request_completed(self,one,two):
        print("request completed")
        print(one)
        print(two)
    
    #this works:
    def request_failed(self,one,two):
        print("request failed")
        print(one)
        print(two)
    
    #this won't work
    def request_failed_wrong(self,one,two,three):
        print("request failed")
        print(one)
        print(two)
        print(three)
    
    def __init__(self, login, password, url, connection):
        self.fx = None
        self.thread = None
        self.status = fxcorepy.AO2GSessionStatus.O2GSessionStatus.DISCONNECTED
        self.connectionSwitchEvent = threading.Event()
        self.login = login
        self.password = password
        self.url = url
        self.connection = connection
        self.thread = threading.Thread(target=self.fcThread)
        self.thread.setDaemon(True)
        self.thread.start()
        self.response_listener = ResponseListener(
            session=self.fx.session,
            on_request_completed_callback = self.request_completed,
            on_request_failed_callback = self.request_failed
        )
        self.response_listener_wrong = ResponseListener(
            session=self.fx.session,
            on_request_completed_callback = self.request_completed,
            on_request_failed_callback = self.request_failed_wrong
        )
    
    def session_status_changed(self, session: fxcorepy.O2GSession,
            status: fxcorepy.AO2GSessionStatus.O2GSessionStatus):
        self.status = status
    
    def fcThread(self):
        self.fx = ForexConnect()
        while True:
            self.fx.login(
                self.login,
                self.password,
                self.url,
                self.connection,
                session_status_callback=self.session_status_changed
            )
            self.connectionSwitchEvent.clear()
            self.connectionSwitchEvent.wait()
            self.fx.logout()
            self.connectionSwitchEvent.clear()
            self.connectionSwitchEvent.wait()
    
    def connect(self):
        if self.status == fxcorepy.AO2GSessionStatus.O2GSessionStatus.DISCONNECTED:
            self.connectionSwitchEvent.set()
    
    def disconnect(self):
        if self.status == fxcorepy.AO2GSessionStatus.O2GSessionStatus.CONNECTED:
            self.connectionSwitchEvent.set()

In [2]:
fc = FC("d161038313","1234","https://www.fxcorporate.com/Hosts.jsp","Demo")
fc.connect()

In [5]:
fc.status

forexconnect.lib.fxcorepy.O2GSessionStatus.CONNECTED

In [6]:
request = fc.fx.create_order_request(
    order_type=fxcorepy.Constants.Orders.TRUE_MARKET_OPEN,
    ACCOUNT_ID="1",
    BUY_SELL="B",
    AMOUNT=1,
    SYMBOL=Common.get_offer(fc.fx,"EUR/USD").instrument
)
fc.fx.send_request_async(request, fc.response_listener)

ERROR:root:Request failed MINIDEMO_7F5BEC95D8867FF8E053E12B3C0A5F25_02082019025531025289_P-23: Description=19915;DAS 19915: ZDas Exception
 ORA-20001: Account is not in pool. Skip request for acct 1



request failed
MINIDEMO_7F5BEC95D8867FF8E053E12B3C0A5F25_02082019025531025289_P-23
Description=19915;DAS 19915: ZDas Exception
 ORA-20001: Account is not in pool. Skip request for acct 1



In [7]:
request = fc.fx.create_order_request(
    order_type=fxcorepy.Constants.Orders.TRUE_MARKET_OPEN,
    ACCOUNT_ID="1",
    BUY_SELL="B",
    AMOUNT=1,
    SYMBOL=Common.get_offer(fc.fx,"EUR/USD").instrument
)
fc.fx.send_request_async(request, fc.response_listener_wrong)

ERROR:root:Request failed MINIDEMO_7F5BEC95D8867FF8E053E12B3C0A5F25_02082019025531025289_P-25: Description=19915;DAS 19915: ZDas Exception
 ORA-20001: Account is not in pool. Skip request for acct 1

